In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [5]:
results = pd.read_csv('mpc_results.csv')
results

,Unnamed: 0,battery_bid_fcas_power,battery_bid_fcas_price,battery_bid_fcas_type,battery_bid_energy_power_gen,battery_bid_energy_power_load,battery_bid_energy_price,battery_bid_energy_type,fcas_clearing_price,energy_price,raise_demand,raise_price,soe,timestamp
0,0,3.624800e-07,41.00,gen,0.000000,300.000000,54.90,load,63.00,54.90,41.00,27.81000,0.000000,2018-06-01 04:30:00
1,1,4.003659e-05,41.00,gen,0.000000,300.000003,56.19,load,63.00,56.19,41.00,28.50000,300.000029,2018-06-01 05:00:00
2,2,2.000000e+02,41.00,gen,0.000000,300.000000,58.73,load,41.00,58.73,41.00,24.97000,600.000078,2018-06-01 05:30:00
3,3,2.000000e+02,41.67,gen,0.000000,281.000201,62.98,load,41.67,62.98,41.67,20.59000,907.999798,2018-06-01 06:00:00
4,4,2.000000e+02,47.67,gen,0.000000,160.000002,74.24,load,47.67,74.24,47.67,18.24996,1029.000000,2018-06-01 06:30:00
5,5,2.000000e+02,75.59,gen,-300.000002,0.000000,229.57,gen,70.00,229.57,75.59,30.00000,1029.000000,2018-06-01 07:00:00
6,6,2.000000e+02,62.14,gen,-109.000269,0.000000,104.33,gen,62.14,104.33,62.14,42.27879,729.000355,2018-06-01 07:30:00
7,7,2.000000e+02,81.00,gen,-299.999996,0.000000,112.42,gen,70.00,112.42,81.00,35.01569,644.913102,2018-06-01 08:00:00
8,8,2.000000e+02,81.00,gen,-184.912588,0.000000,103.99,gen,70.00,103.99,81.00,30.00000,344.912589,2018-06-01 08:30:00
9,9,2.000000e+02,78.92,gen,0.000000,299.999999,94.00,load,70.00,94.00,78.92,20.00000,159.999484,2018-06-01 09:00:00


In [6]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['soe'], name='soe'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['energy_price'], name='energy price'), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Mean of demand and price for a day",
    xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Electricity Demand (MW)", secondary_y=False)
fig.update_yaxes(title_text="Electricity Price ($/MW)", secondary_y=True)

fig.show()

In [7]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['battery_bid_fcas_power'], name='raise power'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['raise_price'], name='raise price'), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Mean of demand and price for a day",
    xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Electricity Demand (MW)", secondary_y=False)
fig.update_yaxes(title_text="Electricity Price ($/MW)", secondary_y=True)

fig.show()

In [8]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['battery_bid_energy_power_load'], name='energy power load'), secondary_y=False)
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['battery_bid_energy_power_gen'], name='energy power gen'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['energy_price'], name='energy price'), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Mean of demand and price for a day",
    xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Electricity Demand (MW)", secondary_y=False)
fig.update_yaxes(title_text="Electricity Price ($/MW)", secondary_y=True)

fig.show()

In [9]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['battery_bid_fcas_price'], name='fcas bid'), secondary_y=False)
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['battery_bid_fcas_power'], name='fcas power'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['fcas_clearing_price'], name='fcas cleared'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['raise_demand'], name='fcas demand'), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Mean of demand and price for a day",
    xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Fcas Price ($/MW)", secondary_y=False)
fig.update_yaxes(title_text="Fcas Demand (MW)", secondary_y=True)

fig.show()